In [16]:
import pandas as pd


In [17]:
#datasets

all_data = "emails.csv"

In [19]:
x = pd.read_csv(all_data).head()
x

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [21]:
import pyspark

In [23]:
from pyspark.sql import SparkSession

In [25]:
spark=SparkSession.builder.appName("Gold").getOrCreate()

In [26]:
spark

In [28]:
## Lets define the dataset column type 
df_pyspark = spark.read.option('header', 'true').csv(all_data, inferSchema=True)

In [29]:
df_pyspark.show(10)

+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|
|           Date: Mon| 14 May 2001 16:3...|
|From: phillip.all...|                null|
|To: tim.belden@en...|                null|
|           Subject: |                null|
|   Mime-Version: 1.0|                null|
|Content-Type: tex...|                null|
|Content-Transfer-...|                null|
|X-From: Phillip K...|                null|
|X-To: Tim Belden ...|                null|
+--------------------+--------------------+
only showing top 10 rows

